In [1]:
from basic_tools import *
from config import DockerConfig
from media import Media
from openai_api import GPTApi, WisperApi
from subtitle import SubtitleWriter, SubtitleDecoder
from media import Media

cfg = DockerConfig()
gpt = GPTApi(cfg("OPENAI_API_KEY_PETER"))
wis = WisperApi(cfg("OPENAI_API_KEY_PETER"))

读取本地配置文件: config.ini


In [2]:

def gpt_split_sentence_translation(gpt: GPTApi, text: str, language: str):

    prompt_translation = """
You are a professional translator proficient in multiple languages. Your task is to translate the text enclosed by <<<>>> into {}, achieving the most natural and fluent translation possible.
Present in the following format:
---
example translated paragraph
---
""".format(language, language)

    prompt_split = """
You are a subtitle segmenter. Your task is to segment the text enclosed in <<<>>> symbols into short sections based on semantics and pauses, with each section not exceeding 12 words and containing no punctuation. Each segmented subtitle should be presented in the following format:
---
example subtitle text 1
---
example subtitle text 2
---
Maintain the natural flow of the dialogue and ensure each segment can independently convey a complete meaning. Do not provide any explanations with your outputs. Do not modify any text. 
    """

    prompt_match2 = """
Your task is to match sentences in two different languages that have the same meaning. The final output should be segmented results in both languages, matched in the specified format.

Text within <<<>>> contains fixed text, segmented by "|".
Text within ((( ))) contains text to be matched, without separators.
Match each segment of the fixed text with the most accurate segment from the text to be matched, ensuring the best possible meaning match. Ensure a one-to-one correspondence as much as possible. Match the text to be matched in sequence. Each fixed text segment must be matched with one segment from the text to be matched, leaving no segments unmatched.

Each segment should be presented in the following format:
---
example fixed text 1
-
example matched text 1
---
example fixed text 2
-
example matched text 2
---
Please do not provide any explanations and do not answer any questions.
"""

    prompt_match = """
Your task is to match sentences in two different languages that have the same meaning. The final output should be segmented results in both languages, matched in the specified format.

Text within <<<>>> contains fixed text, segmented by "|".
Text within ((( ))) contains text to be matched, without separators.
Match each segment of the fixed text with the most accurate segment from the text to be matched, ensuring the best possible meaning match. Multiple segments can be matched with each other, but aim for one-to-one correspondence as much as possible. Try to match the text to be matched in sequence.

Each segment should be presented in the following format:
---
example fixed text 1
-
example matched text 1
---
example fixed text 2
-
example matched text 2
---
Please do not provide any explanations and do not answer any questions.
"""
    success = False
    #print("DEBUG 原文: ", text)
    for i in range(5):
        try:
            usermsg = "<<<{}>>>".format(text)
            message = [
                {"role": "system", "content": prompt_translation},
                {"role": "user", "content": usermsg}
            ]
            response = gpt.query(message, max_tokens=4000, temperature=0.2, model="gpt-4o", timeout=120)
            sentences = response[4:-4]
            #print("DEBUG stc: ", sentences)

            usermsg = "<<<{}>>>".format(sentences)
            message = [
                {"role": "system", "content": prompt_split},
                {"role": "user", "content": usermsg}
            ]
            response = gpt.query(message, max_tokens=4000, temperature=0, model="gpt-4o", timeout=120)
            sentences = response[4:-4].split("\n---\n")
            #print("DEBUG stc: ", sentences)

            usermsg = "<<<{}>>> \n((({})))".format("|".join(sentences), text)
            message = [
                {"role": "system", "content": prompt_match},
                {"role": "user", "content": usermsg}
            ]
            response = gpt.query(message, max_tokens=4000, temperature=0, model="gpt-4o", timeout=120)
            sentences = response[4:-4].split("\n---\n")
            #print("DEBUG stc: ", sentences)
            sentences = [x.split("\n-\n") for x in sentences]
            translation = [x[0] for x in sentences]
            match_only = [x[1] for x in sentences]

            success = True
            break
        except Exception as e:
            print("[gpt_split_sentence_translation]: GPT-4o Invalid Response.")
            print("[gpt_split_sentence_translation]: Retry {} times.".format(i+1))
            print("[gpt_split_sentence_translation]: Error : {}".format(str(e)))
            continue
    
    if not success:
        print("[gpt_split_sentence_translation]: GPT-4o Invalid Response.")
        print("[gpt_split_sentence_translation]: Please Check Your Network.")
        print("[gpt_split_sentence_translation]: Exit.")
        input("Press Enter to Exit...")
        exit(0)

    # print("DEBUG 原文: ", text)
    # print("DEBUG 翻译: ", translation)
    # print("DEBUG 匹配: ", match_only)

    return translation, match_only

In [3]:
pathFile = FilePath(r"C:\Users\lucyc\Videos\Captures\【日韓交流】５年ぶりの対面交流会　“歴史問題”苦い思い出も　愛知・刈谷市 - YouTube - Google Chrome 2024-05-25 05-08-41.mp4")

In [4]:
md = Media(pathFile,
           SystemCmd())

In [5]:
md.split_start = 0
res = md.get_audio_pice(120)

In [6]:
audio_file = open(pathFile.nfile(full_name="temp.mp3")(), "rb")
transcript = wis.transcribe_timestamp(audio_file)

In [7]:
transcript

Transcription(text='現れた時代のバス カリア市内の高校に現れた時代のバス 生徒たちが興奮した様子で出迎えていたのは韓国の高校生たちです 改めて紹介します韓国観光高校のみなさんです 様々な企画をみんなに楽しんでもらえるように 用意したのでぜひ楽しんでいってもらえると嬉しいです およそ11年前から高校生が加害の学校を定期的に訪問するこの交流会 コロナの影響などもあり5年ぶりに対面での交流となりました 韓国で日本語を勉強している生徒たち 共通の話題で盛り上がっていたのは 日本の部活動体験では作動部でお茶のたて方などを体験していたのですが 日本語を勉強している生徒たち', task='transcribe', language='japanese', duration=120.0, words=[{'word': '現', 'start': 0.47999998927116394, 'end': 0.9200000166893005}, {'word': 'れた', 'start': 0.9200000166893005, 'end': 0.46000000834465027}, {'word': '時', 'start': 0.46000000834465027, 'end': 1.3600000143051147}, {'word': '代', 'start': 1.3600000143051147, 'end': 1.5800000429153442}, {'word': 'の', 'start': 1.5800000429153442, 'end': 3.859999895095825}, {'word': 'バ', 'start': 3.859999895095825, 'end': 4.659999847412109}, {'word': 'ス', 'start': 4.659999847412109, 'end': 4.659999847412109}, {'word': 'カ', 'start': 4.659999847412109, 'end': 7.179999828338623}, {'word': 'リ', 'start': 7.179999828338623, 'end': 7.199999809265137}, {'word'

In [8]:
transcript.words

[{'word': '現', 'start': 0.47999998927116394, 'end': 0.9200000166893005},
 {'word': 'れた', 'start': 0.9200000166893005, 'end': 0.46000000834465027},
 {'word': '時', 'start': 0.46000000834465027, 'end': 1.3600000143051147},
 {'word': '代', 'start': 1.3600000143051147, 'end': 1.5800000429153442},
 {'word': 'の', 'start': 1.5800000429153442, 'end': 3.859999895095825},
 {'word': 'バ', 'start': 3.859999895095825, 'end': 4.659999847412109},
 {'word': 'ス', 'start': 4.659999847412109, 'end': 4.659999847412109},
 {'word': 'カ', 'start': 4.659999847412109, 'end': 7.179999828338623},
 {'word': 'リ', 'start': 7.179999828338623, 'end': 7.199999809265137},
 {'word': 'ア', 'start': 7.199999809265137, 'end': 7.300000190734863},
 {'word': '市', 'start': 7.300000190734863, 'end': 7.480000019073486},
 {'word': '内', 'start': 7.480000019073486, 'end': 7.679999828338623},
 {'word': 'の', 'start': 7.679999828338623, 'end': 8.15999984741211},
 {'word': '高', 'start': 8.15999984741211, 'end': 8.239999771118164},
 {'word':

In [25]:
t, m = gpt_split_sentence_translation(gpt, transcript.text, "Chinese")

In [26]:
m

['日本の部活動体験では',
 '作動部でお茶の立て方などを体験していたのですが',
 'こうした体験や生徒同士が積極的にコミュニケーションを取ることが',
 'この交流会の狙いだといいます',
 '日本と韓国の若い世代が',
 'お互いの国を考える交流会',
 'そして今、トップ同士の交流では',
 '日韓関係が大きく動き出しています',
 '日本と韓国の両首脳は',
 'お互いの国を訪れるシャトル外交を12年ぶりに再開',
 'そこで、いわゆる元徴用工問題などの歴史問題の解決よりも',
 '未来に向けた協力を優先すべきと発言しました',
 '未来に向けた協力を優先すべきと発言しました',
 '今回の交流会に参加したカリアキタ高校のヨツハさん',
 '家族で韓国に行ったとき',
 '現地の人たちに']

In [30]:

def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]


def sliding_matching(words_data, sentences):
    # remove all spaces
    sentences = [x.replace(" ", "") for x in sentences]
    words = []
    words_map = []
    time_map = []
    location = 0
    
    for i, word in enumerate(words_data):
        wordStr = word["word"]
        for char in wordStr:
            words.append(char)
            words_map.append(i)
    

    for sentence in sentences:
        min_distance = 9999999
        min_location = 0

        for i in range(0, len(words)-len(sentence)+1):
            be_matched = "".join(words[i:i+len(sentence)])
            distance = levenshtein_distance(be_matched, sentence)
            distance += abs(i - location) * 0.1

            #print("be_matched: {} distance: {}, len {}".format(be_matched, distance, len(be_matched)))
            if distance < min_distance:
                min_distance = distance
                min_location = i

        location += len(sentence)
        # print("\n\nmin_location: ", min_location)
        # print("sentence: ", sentence)
        # print("Matched : ", "".join(words[min_location:min_location+len(sentence)]))
        time_map.append(
            (words_data[words_map[min_location]]["start"],
             words_data[words_map[min_location + len(sentence)-1]]["end"])
        )
    
    return time_map


SyntaxError: invalid syntax (2119206429.py, line 62)

In [31]:
time_map = sliding_matching(transcript.words, m)


In [29]:
time_map


[(0.0, 2.140000104904175),
 (2.680000066757202, 7.480000019073486),
 (30.0, 34.720001220703125),
 (35.29999923706055, 37.29999923706055),
 (57.52000045776367, 60.34000015258789),
 (60.599998474121094, 62.900001525878906),
 (64.9000015258789, 67.45999908447266),
 (67.68000030517578, 70.44000244140625),
 (71.91999816894531, 73.91999816894531),
 (74.05999755859375, 78.13999938964844),
 (81.18000030517578, 86.37999725341797),
 (86.44000244140625, 89.33999633789062),
 (86.44000244140625, 89.33999633789062),
 (112.36000061035156, 116.55999755859375),
 (117.76000213623047, 119.31999969482422),
 (119.66000366210938, 119.9800033569336)]